# Self-study try-it 17.3: Evaluate and Explain CNN Performance

In this activity you will evaluate the CNN Performance using confusion matrix.

In [ ]:
%matplotlib inline

## Training an image classifier
----------------------------

In continuation with our previous Self-study Try-it activities 17.1, 17.2, we will first train the image classifier using the CIFAR10 dataset and then evaluate the CNN's performance using a confusion matrix.
1. Load and normalise the CIFAR10 training and test data sets using
   ``torchvision``.
2. Define a convolutional neural network.
3. Define a loss function.
4. Train the network on the training data.
5. Test the network on the test data.

### Step 1: Loading and normalising CIFAR10


Using ``torchvision``, it’s extremely easy to load CIFAR10.



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

The output of torchvision data sets are PILImage images of range [0, 1].
We transform them to Tensors of normalised range [-1, 1].



In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Let us show some of the training images (just for fun!).



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter) # Changed dataiter.next() to next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

### Step 2: Define a convolutional neural network


Copy the neural network from the neural networks section before and modify it to
take 3-channel images (instead of 1-channel images as it was defined).



In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, name=None):
        super(Net, self).__init__()
        if name:
            self.name = name
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net(name='LetNet5')

### Step 3: Define a loss function and optimiser

Let's use a Classification Cross-Entropy loss and SGD with momentum.



In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Step 4: Train the network


This is when things start to get interesting.
We simply have to loop over our data iterator and feed the inputs to the
network and optimise. We will also use the `time` package to get the training time of the network.



In [ ]:
import time

start = time.time()

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

end = time.time()
print('training time ', end-start)

### Step 5: Test the network on the test data


We have trained the network for 2 passes over the training data set, but we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network
outputs and then comparing it against the ground-truth. If the prediction is
correct, we add the sample to the list of correct predictions.

First step: Let us display an image from the test set to get familiar.



In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter) # Use next(dataiter) instead of dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

Now let us see what the neural network thinks these examples are.



In [ ]:
outputs = net(images)

The outputs are energies for the 10 classes.
The higher the energy for a class, the more the network
thinks that the image is of the particular class.
So, let's get the index of the highest energy:



In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

The results seem pretty good.

Let us look at how the network performs on the whole data set.



In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

That looks way better than chance, which is 10% accuracy (randomly picking
a class out of 10 classes).
It seems like the network learnt something!

Now let's answer the following question: What are the classes that performed well and the classes that did
not?



In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

### Step 6: Evaluate with Confusion Matrix and Per-Class Accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd

def evaluate_model(net, dataloader, classes):
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap='Blues', xticks_rotation='vertical')
    plt.title("Confusion Matrix")
    plt.show()

    # Per-class accuracy
    cm_df = pd.DataFrame(cm, index=classes, columns=classes)
    correct_per_class = cm_df.values.diagonal()
    total_per_class = cm_df.sum(axis=1).values
    accuracy_per_class = correct_per_class / total_per_class

    print("\nPer-class Accuracy:")
    for i, acc in enumerate(accuracy_per_class):
        print(f"{classes[i]:>10}: {acc*100:.2f}%")

# Run evaluation
evaluate_model(net, testloader, classes)


### Step 7: Visualize Misclassified Images

In [ ]:
def show_misclassified(net, dataloader, classes, num_images=5):
    misclassified = []

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            for i in range(len(labels)):
                if predicted[i] != labels[i]:
                    misclassified.append((images[i], predicted[i], labels[i]))
                if len(misclassified) >= num_images:
                    break
            if len(misclassified) >= num_images:
                break

    # Display
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    for idx, (img, pred, label) in enumerate(misclassified):
        img = img / 2 + 0.5  # unnormalize
        npimg = img.numpy()
        axes[idx].imshow(np.transpose(npimg, (1, 2, 0)))
        axes[idx].set_title(f"Pred: {classes[pred]}\nTrue: {classes[label]}")
        axes[idx].axis('off')
    plt.tight_layout()
    plt.show()

# Show misclassified samples
show_misclassified(net, testloader, classes)


###  Reflection Prompts

- Which classes did the model struggle with most, and what might explain these errors?
- How could you modify the network architecture or training strategy to improve performance?
- What insights do the misclassified images offer about the limitations of your model?


### Question 1: Which classes did the model struggle with most, and what might explain these errors?

The lowest per class accuracies are Cat, Deer, Bird with accuracies of 30.5%, 35.4%, 38.7% respectively.

This can be because of **visual ambiguity**, because they often share similar textures, or shapes at low resolution(32x32) making them harder to distinguish.

**Intra-class variability**: Classes like “cat” and “bird” have high diversity in poses, colors, and backgrounds, which the model may not generalize well across.

**Limited depth**: The current CNN (LetNet5 variant) is relatively shallow and may not extract sufficiently abstract features for nuanced distinctions.

### Question 2: How could you modify the network architecture or training strategy to improve performance?

Here are a few targeted improvements:

**Architectural Enhancements:**

- Add more convolutional layers: Deeper networks like ResNet or VGG can capture more complex patterns.

- Use Batch Normalization: Helps stabilize and accelerate training.

- Replace fully connected layers with Global Average Pooling: Reduces overfitting and improves spatial generalization.

**Training Strategy Tweaks:**

- Data Augmentation: Apply random crops, flips, rotations, and color jitter to improve robustness.

- Train longer or with learning rate scheduling: Two epochs is minimal—try 20–50 with a decaying learning rate.

- Use Adam optimizer: Often converges faster and more reliably than SGD with momentum.

### Question 3: What insights do the misclassified images offer about the limitations of your model?

- **Semantic confusion**: The model confuses animals with similar shapes or contexts (e.g., dog vs. cat), indicating limited feature abstraction.

- **Background bias**: CIFAR-10 images often contain cluttered backgrounds, which may mislead the model if it overfits to context rather than object.

- **Resolution bottleneck:** At 32×32, fine-grained details are lost—this limits the model’s ability to distinguish subtle features like ears or wings.